In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 44.6 MB/s 
     |████████████████████████████████| 9.3 MB 8.0 MB/s 
     |████████████████████████████████| 108 kB 45.4 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [10]:
df_5percent=df[0:2344] 


In [11]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,3,1,1,1,...,1,0,3,0,7.235317,7.954019,248832,7.507871,0,1
1,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,5.630848,0.299051,10752,4.924023,1,0
2,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.329041,2.719303,181760,6.824579,1,0
3,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,5.119311,3.548546,70144,4.954553,1,0
4,144,3,4,65535,184,216,3,1,0,1,...,1,0,3,0,7.284032,7.944639,251904,7.529611,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,5.509337,0.299051,36352,4.165316,1,0
2340,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.484293,7.748603,140800,7.328279,1,1
2341,144,3,4,65535,184,232,5,1,0,1,...,1,2,3,0,6.474336,3.266392,768000,6.438159,1,0
2342,144,3,4,65535,184,240,5,1,0,1,...,1,2,3,0,6.369398,2.458533,921600,6.416433,1,0


In [12]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1103, 69)
class 1: (1241, 69)


In [13]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [14]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     232.0               3.0   
1      144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
4      144.0   3.0        4.0     65535.0  184.0     216.0               3.0   
...      ...   ...        ...         ...    ...       ...               ...   
2339   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
2340   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
2341   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
2342   144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
2343   144.0   3.0        4.0     65535.0  184.0     176.0               2.0   

      C

In [15]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.0,...,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2344.000000,2.344000e+03,2344.000000,2344.000000,2344.000000
mean,138.887372,2.912116,3.973549,65199.496587,183.003413,223.177474,4.655290,0.992747,0.376280,1.0,...,0.999147,1.337457,3.317833,0.154437,4.973002,2.583820,8.329993e+05,6.382110,0.530717,0.529437
std,19.056471,0.343152,0.321621,4678.032685,15.920952,46.343658,1.924024,0.084871,0.484555,0.0,...,0.029204,1.639263,1.112190,0.361444,2.505417,2.728646,5.263910e+06,1.112661,0.499162,0.499239
min,0.000000,0.000000,0.000000,0.000000,0.000000,64.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942896,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,4.293493,0.000000,6.131200e+04,5.700151,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.173011,1.654012,1.218560e+05,6.407426,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.504981,4.709637,3.115020e+05,7.345447,1.000000,1.000000
max,144.000000,4.000000,4.000000,65535.000000,320.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,7.000000,1.000000,7.999070,7.997932,1.223571e+08,7.999997,1.000000,1.000000


In [16]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [17]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [18]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [19]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      60.0
0.0      40.0
dtype: float64

In [20]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [21]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [22]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [23]:
circuit(weights_init, data[0])

tensor(0.09270431, requires_grad=True)

In [24]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [25]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [26]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [27]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [28]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.25324611, requires_grad=False), tensor(-0.24175815, requires_grad=False)], Y =  1
X = [tensor(-0.24244103, requires_grad=False), tensor(-0.31194387, requires_grad=False)], Y =  1
X = [tensor(-0.24368061, requires_grad=False), tensor(-0.3039377, requires_grad=False)], Y =  1
X = [tensor(-0.24156311, requires_grad=False), tensor(-0.31805805, requires_grad=False)], Y =  1
X = [tensor(-0.32473058, requires_grad=False), tensor(-0.34630726, requires_grad=False)], Y = -1


In [29]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [30]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 0.9365066 | Accuracy: 0.6500000 
Iter:     2 | Cost: 0.9122814 | Accuracy: 0.6187500 
New best
Iter:     3 | Cost: 0.9203170 | Accuracy: 0.7062500 
Iter:     4 | Cost: 1.1463826 | Accuracy: 0.4625000 
Iter:     5 | Cost: 1.2145558 | Accuracy: 0.4625000 
Iter:     6 | Cost: 0.9869825 | Accuracy: 0.4500000 
Iter:     7 | Cost: 0.9084670 | Accuracy: 0.6812500 
Iter:     8 | Cost: 0.8925828 | Accuracy: 0.6250000 
Iter:     9 | Cost: 0.8922195 | Accuracy: 0.6250000 
Iter:    10 | Cost: 0.9133098 | Accuracy: 0.7062500 
Iter:    11 | Cost: 0.9312235 | Accuracy: 0.6875000 
Iter:    12 | Cost: 0.9395710 | Accuracy: 0.6000000 
Iter:    13 | Cost: 0.9182165 | Accuracy: 0.7000000 
Iter:    14 | Cost: 0.9015893 | Accuracy: 0.6625000 
Iter:    15 | Cost: 0.8955067 | Accuracy: 0.6375000 
Iter:    16 | Cost: 0.9133180 | Accuracy: 0.6250000 
Iter:    17 | Cost: 0.9384979 | Accuracy: 0.6250000 
Iter:    18 | Cost: 0.9972926 | Accuracy: 0.6187500 
Iter:    19 | Cost: 0.991609

In [31]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [32]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.8225888664169375, Accuracy: 72.0%


In [33]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,-1.0,-1.0
2,1.0,-1.0
3,1.0,-1.0
4,1.0,1.0
5,1.0,-1.0
6,1.0,1.0
7,-1.0,-1.0
8,1.0,1.0
9,1.0,1.0


In [34]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.47      0.90      0.62        10
         1.0       0.95      0.67      0.78        30

    accuracy                           0.73        40
   macro avg       0.71      0.78      0.70        40
weighted avg       0.83      0.72      0.74        40

0.9523809523809523
0.6666666666666666
0.7843137254901961
0.7833333333333333
